# NYC Taxi Fare Prediction

Predict taxi fares using the [New York City Taxi and Limousine Commission (TLC) Trip Record Data](https://registry.opendata.aws/nyc-tlc-trip-records-pds/) public dataset.

In [ ]:
%%capture
!pip install -U pandas geopandas seaborn

## Data Prep
 
In this section of the notebook, you will download the publicly available New York Taxi dataset in preparation for uploading it to S3.

### Download Dataset

First, download a sample of the New York City Taxi [dataset](https://registry.opendata.aws/nyc-tlc-trip-records-pds/)⇗ to this notebook instance. 

This dataset contains information on trips taken by taxis and for-hire vehicles in New York City, including pick-up and drop-off times and locations, fares, distance traveled, and more. 

In [ ]:
!aws s3 cp 's3://nyc-tlc/trip data/green_tripdata_2018-02.csv' 'nyc-tlc.csv'

In [ ]:
!aws s3 cp 's3://nyc-tlc/misc/taxi_zones.zip' 'taxi_zones.zip'

In [ ]:
!unzip taxi_zones.zip -d shapes

### Load Datasets

Load the trip dataset

In [ ]:
import pandas as pd

trip_df = pd.read_csv(
    "nyc-tlc.csv", parse_dates=["lpep_pickup_datetime", "lpep_dropoff_datetime"]
)
trip_df.head()

Load the taxi zone shape data to get the gemotry and calculate a centroid and lat/long  each location

In [ ]:
import geopandas as gpd

# Load the shape file and get the geometry and lat/lon
zones = gpd.read_file("shapes/taxi_zones.shp")
# Return Centroid as CRS code of 3310 for calcuating distance in meters.
zones["centroid"] = zones.geometry.centroid.to_crs(epsg=3310)
# Convert cordinates to the WSG84 lat/long CRS has a EPSG code of 4326.
zones["latitude"] = zones.centroid.to_crs(epsg=4326).x
zones["longitude"] = zones.centroid.to_crs(epsg=4326).y

# Drop duplicate by location ID keeping the first
zones = zones.drop_duplicates(subset="LocationID", keep="first")
# Drop cols we don't need and inspect results
zones = zones.set_index("LocationID").drop(
    ["OBJECTID", "Shape_Leng", "Shape_Area"], axis=1
)
zones.head()

Join the trip data to the zone and calculate the distance between centroids (should take < 20 seconds)

In [ ]:
%%time
trip_df = gpd.GeoDataFrame(
    trip_df.join(zones, on="PULocationID").join(
        zones, on="DOLocationID", rsuffix="_DO", lsuffix="_PU"
    )
)
trip_df["geo_distance"] = trip_df["centroid_PU"].distance(trip_df["centroid_DO"]) / 1000
trip_df[["PULocationID", "DOLocationID", "trip_distance", "geo_distance"]].head()

Add datetime parts based on pickup time and duration to validate results

In [ ]:
trip_df["hour"] = trip_df.lpep_pickup_datetime.dt.hour
trip_df["weekday"] = trip_df.lpep_pickup_datetime.dt.weekday
trip_df["month"] = trip_df.lpep_pickup_datetime.dt.month
trip_df["duration_minutes"] = (
    trip_df["lpep_dropoff_datetime"] - trip_df["lpep_pickup_datetime"]
).dt.seconds / 60

### Data visualization

Let's check that we have a good spread of travel across each day of the week and hour of the day

In [ ]:
import seaborn as sns

sns.histplot(trip_df, x="hour")

And plot that we have a distribution across week days

In [ ]:
sns.histplot(trip_df, x="weekday")

Let's validate that the geo distance correlations generally with the fare amount

In [ ]:
sample_df = trip_df[trip_df["geo_distance"] > 0].sample(1000)
sns.jointplot(data=sample_df, x="geo_distance", y="fare_amount")

Plot the geometry of the map along with centroids for each location

In [ ]:
import matplotlib.pyplot as plt
from shapely.geometry import LineString


def annotate(ax, z):
    txt = f"{z.name}: {z.zone} ({-z.latitude:.2f}°N, {z.longitude:.2f}°W)"
    ax.annotate(txt, (z.latitude, z.longitude))


def arrow(ax, ll):
    ld = ll.iloc[1] - ll.iloc[0]
    ax.arrow(
        ll.iloc[0].latitude,
        ll.iloc[0].longitude,
        ld.latitude,
        ld.longitude,
        length_includes_head=True,
        edgecolor="lightgrey",
    )


def plot_map(zones, zids):
    # Render the geometry in Lat/Lon space
    ax = zones.geometry.to_crs(epsg=4326).plot(
        figsize=(15, 15), color="whitesmoke", edgecolor="lightgrey", linewidth=0.5
    )
    # Draw arrow
    arrow(ax, zones.loc[zids][["latitude", "longitude"]])
    # Plot centroid
    centroids = zones.loc[zids].geometry.centroid.to_crs(
        epsg=3310
    )  # Require this format for calculating distance
    centroids.to_crs(epsg=4326).plot(ax=ax, color="red", marker="+")
    # Annotate points
    for i, row in zones.loc[zids].iterrows():
        annotate(ax, row)
    # Output the distance traveled
    dist = centroids.iloc[0].distance(centroids.iloc[1]) / 1000
    plt.title(f"From zone {zids[0]} to {zids[1]} distance: {dist:.2f}km")
    return dist

Select a trip to inspect the zones it travels from and to and the duration and cost

In [ ]:
trip_idx = 5

# Get the trip and plot on map
t = trip_df.iloc[trip_idx]
dist = plot_map(zones, [t.PULocationID, t.DOLocationID])

print(
    f"Took {t.duration_minutes:.2f} minutes on {t.weekday} at {t.hour} hour to travel {dist:.2f}km for the cost of ${t.fare_amount:.2f}"
)

### Feature selection

Rename and select columns that we want build model on

In [ ]:
# Rename cols
trip_df = trip_df.rename(
    columns={
        "latitude_PU": "pickup_latitude",
        "longitude_PU": "pickup_longitude",
        "latitude_DO": "dropoff_latitude",
        "longitude_DO": "dropoff_longitude",
    }
)

# Select cols
cols = [
    "fare_amount",
    "pickup_latitude",
    "pickup_longitude",
    "dropoff_latitude",
    "dropoff_longitude",
    "geo_distance",
    "hour",
    "weekday",
    "month",
]
data_df = trip_df[cols]
data_df.sample(5)

Clean up to remove some outliers

In [ ]:
data_df = data_df[
    (data_df.fare_amount > 0)
    & (data_df.fare_amount < 200)
    & (data_df.geo_distance >= 0)
    & (data_df.geo_distance < 121)
].dropna()
print(data_df.shape)

### Data splitting and saving


We are now ready to split the dataset into train, validation, and test sets. 

In [ ]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(data_df, test_size=0.20, random_state=42)
val_df, test_df = train_test_split(val_df, test_size=0.05, random_state=42)

# Reset the index for our test dataframe
test_df.reset_index(inplace=True, drop=True)

print(
    "Size of\n train: {},\n val: {},\n test: {} ".format(
        train_df.shape[0], val_df.shape[0], test_df.shape[0]
    )
)

Save the train, validation, and test files as CSV locally on this notebook instance. Notice that you save the train file twice - once as the training data file and once as the baseline data file. The baseline data file will be used by [SageMaker Model Monitor](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor.html)⇗ to detect data drift. Data drift occurs when the statistical nature of the data that your model receives while in production drifts away from the nature of the baseline data it was trained on, which means the model begins to lose accuracy in its predictions.

In [ ]:
train_df.to_csv("train.csv", index=False, header=False)
val_df.to_csv("validation.csv", index=False, header=False)
test_df.to_csv("test.csv", index=False, header=False)

# Save test and baseline with headers
train_df.to_csv("baseline.csv", index=False, header=True)

Now upload these CSV files to your default SageMaker S3 bucket. 

In [ ]:
import sagemaker

# Get the session and default bucket
session = sagemaker.session.Session()
bucket = session.default_bucket()

# Specify data prefix and version
prefix = "nyc-tlc/v2"

s3_train_uri = session.upload_data("train.csv", bucket, prefix + "/data/training")
s3_val_uri = session.upload_data("validation.csv", bucket, prefix + "/data/validation")
s3_test_uri = session.upload_data("test.csv", bucket, prefix + "/data/test")
s3_baseline_uri = session.upload_data("baseline.csv", bucket, prefix + "/data/baseline")

### Training Job

Build an estimator to train on this, see if using geo_distance its okay predictor.

In [ ]:
# TODO: Can XGBoost report use a version which accepts the header for feature importance?
from sagemaker.estimator import Estimator
from sagemaker.debugger import Rule, rule_configs

# Get role and region
role = sagemaker.get_execution_role()
region = sagemaker.session.Session().boto_session.region_name

# Define the XGBoost training report rules
# see: https://docs.aws.amazon.com/sagemaker/latest/dg/debugger-training-xgboost-report.html
rules = [Rule.sagemaker(rule_configs.create_xgboost_report())]

# Get the training instance type
training_instance_type = "ml.m4.xlarge"

# training step for generating model artifacts
image_uri = sagemaker.image_uris.retrieve(
    framework="xgboost",
    region=region,
    version="1.2-2",
    py_version="py3",
    instance_type=training_instance_type,
)

output_path = "s3://{}/{}/output".format(bucket, prefix)

estimator = Estimator(
    image_uri=image_uri,
    instance_type=training_instance_type,
    instance_count=1,
    output_path=output_path,
    role=role,
    disable_profiler=True,  # Profile processing job
    rules=rules,  # Report processing job
)

hp = {
    "max_depth": "9",
    "eta": "0.2",
    "gamma": "4",
    "min_child_weight": "300",
    "subsample": "0.8",
    "objective": "reg:squarederror",  # reg:linear not supported
    "early_stopping_rounds": "10",
    "num_round": "100",
}
estimator.set_hyperparameters(**hp)

# Set the data
s3_input_train = sagemaker.inputs.TrainingInput(
    s3_data=s3_train_uri, content_type="text/csv"
)
s3_input_val = sagemaker.inputs.TrainingInput(
    s3_data=s3_val_uri, content_type="text/csv"
)
data = {"train": s3_input_train, "validation": s3_input_val}

estimator.fit(data)

### Evaluate 

Wait for the XGBoost report to be ready

In [ ]:
sm_client = sagemaker.session.Session().sagemaker_client

# Attach the job and get report
xgb_report_job_name = [
    rule["RuleEvaluationJobArn"].split("/")[-1]
    for rule in estimator.latest_training_job.rule_job_summary()
    if "CreateXgboostReport" in rule["RuleConfigurationName"]
][0]

print(f"Waiting for XGBoost training report {xgb_report_job_name} to complete...")
sm_client.get_waiter("processing_job_completed_or_stopped").wait(
    ProcessingJobName=xgb_report_job_name
)
print("Done")

Inspects the results of the report

In [ ]:
from IPython.display import FileLink
from sagemaker.s3 import S3Downloader, S3Uploader

# Get the s3 output
report_uri = sm_client.describe_processing_job(ProcessingJobName=xgb_report_job_name)[
    "ProcessingOutputConfig"
]["Outputs"][0]["S3Output"]["S3Uri"]

# Download the notebook from the report
S3Downloader().download(f"{report_uri}/xgboost_report.html", "report")
FileLink("report/xgboost_report.html", result_html_prefix="Open Report: ")

### Deploy

Deploy an endpoint for the predictor

In [ ]:
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import CSVDeserializer

predictor = estimator.deploy(
    initial_instance_count=1,
    instance_type="ml.m4.xlarge",
    serializer=CSVSerializer(),
    deserializer=CSVDeserializer(),
)

Get predictions for the held out test dataset

In [ ]:
def chunker(seq, batch_size):
    return (seq[pos : pos + batch_size] for pos in range(0, len(seq), batch_size))


# Make predictions without the first colunns
results = []
for df in chunker(test_df[test_df.columns[1:]], 20):
    results += predictor.predict(data=df.to_csv(index=False, header=False))[0]

# Get the fare amoiunt pred back in the dataframe\
predictions = pd.Series(results).astype(float)

Join the predictions back to the test dataset

In [ ]:
pred_df = pd.DataFrame({"fare_amount_prediction": predictions})
pred_df = test_df.join(pred_df)

# Get abs error
pred_df["error"] = abs(pred_df["fare_amount"] - pred_df["fare_amount_prediction"])
pred_df.sort_values("error", ascending=False).head()

Calculate the root mean squre error (RMSE) to evaluate the performance of this model. 

In [ ]:
from math import sqrt
from sklearn.metrics import mean_squared_error


def rmse(pred_df):
    return sqrt(
        mean_squared_error(pred_df["fare_amount"], pred_df["fare_amount_prediction"])
    )


print("RMSE: {}".format(rmse(pred_df)))

Plot the residules to see where the errors are relative to the fare amount.

In [ ]:
sns.residplot(
    x=pred_df["fare_amount"], y=pred_df["fare_amount_prediction"], lowess=True
)